In [ ]:
# import settings and functions
%run ./../../imports.ipynb


## What mesh?

Copy your choice to the next cell

for SquareTop:
```
analytical_solution_tag = "-ana_square_top"
generate_config = generateConfig_squareTop
generate_mesh = generateMesh_squareTop
```

for SquareSinCos:
```
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos
```

In [2]:
# Change according to instruction above
analytical_solution_tag = "-ana_square_sincos"
generate_config = generateConfig_squareSinCos
generate_mesh = generateMesh_squareSinCos

# analytical_solution_tag = "-ana_L_shape"
# generate_config = generateConfig_Lshape
# generate_mesh = generateMesh_Lshape

## Analysis setup

In [3]:
# Convergence analysis parameters
order_list = [1, 2, 3, 4] # approximation order p
elem_size_list = [0.2, 0.1, 0.05, 0.02, 0.01] # element size h
# elem_size_list = [0.5, 0.2, 0.1] # element size h
params.triangle_mesh = True

ana_name = "ana_square_top"
sumanalys = "FEM_errors.csv"

run_test = True
run_analysis = True

naming = ["order", "gaussnum", "iterations","volume", "datanum","rmsPoiErr", "errorIndicator",
          "L2norm", "H1seminorm","fluxErr", "orderRefinementCounter"]

error_name_list = ["L2norm", "H1seminorm", "fluxErr"]
error_label_list = [(r'Global error $L^2$-norm'),
               (r'Global error $H^1$-seminorm'), (r'Global Flux error')]

In [4]:
params.conductivity = 1.0 # linear conductivity
params.element_size = 0.1 # element size in the regular mesh
params.order = 1 # approximation order for displacements

# params.triangle_mesh = False # use triangular mesh

# Pre-processing parameters
params.mesh_file = "square_top"
params.length_x = 1
params.length_y = 1
params.length_z = 0
params.show_mesh = True


# solution parameters
params.log_file = "log" # log file name 
params.nproc = 1 # number of processors


## Run test

In [ ]:
# start display for showing results
display = Display(backend="xvfb", visible=False, size=(1024, 768))
display.start()

In [ ]:
# Testing mesh generation
if run_test:
    params.show_mesh = True
    generate_config(params)
    generate_mesh(params)

In [ ]:
# Testing running analysis
if run_test:
    params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
    !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
    !{classic_diffusion} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}

    !convert.py out*


In [ ]:
if run_test:
    params.show_file = "out_ori_result"
    params.show_field = "P_reference"
    params.show_edges = True
    show_results(params)

## Convergence analysis

In [ ]:
if run_analysis:
    !rm {sumanalys}
    !rm ./output_files/out*
    for elem_size in elem_size_list:
        params.element_size = elem_size
        params.show_mesh = False
        generate_mesh(params)
        params.part_file = params.mesh_file + "_" + str(params.nproc) + "p.h5m"
        !{mofem_part} -my_file {params.mesh_file + ".h5m"} -my_nparts {params.nproc} -output_file {params.part_file} -dim 2 -adj_dim 1
        for order in order_list:
            params.order = order
            !{classic_diffusion} -file_name {params.part_file} -my_order {params.order} {analytical_solution_tag}
    !mv {sumanalys} {ana_name}.csv


## Read and organise results

In [10]:
# read analysis results from ana_name
analysis = []
data = pd.read_csv(ana_name+'.csv', header=None,
                   names=naming,  index_col=False)
analysis.append(data)
analysis[0] = data

In [11]:
# separate by order
classic = []

for i in order_list:
    classic.append(analysis[0].query('order == ' + str(i)))

## Plot results

In [ ]:
for (error_name, error_label) in zip(error_name_list,error_label_list):
    fig = plt.figure()
    ax = plt.axes()
    for i in range(len(order_list)):
        x = classic[i]["gaussnum"]
        y = classic[i][str(error_name)]

        line, = ax.plot(x, y, '-x', label=(r'$p$ = ' + str(order_list[i])))

        # Calculate the gradient of the line
        gradient, _ = np.polyfit(np.log(x), np.log(y), 1)

        # Annotate the plot with the gradient value
        ax.annotate('Gradient = {:.1f}'.format(gradient), 
                    xy=(x.iloc[len(x)//2], y.iloc[len(y)//2]), 
                    xycoords='data',
                    xytext=(1, 0),  # move a bit to the right
                    textcoords='offset points'
                    )
        ax.set_xscale('log')
        ax.legend(loc='best')
        ax.set_yscale('log')
        ax.set_ylabel(error_label)
        ax.set_xlabel("Number of integration points")
        ax.grid(True, ls=':')

    plt.tight_layout()
    plt.savefig('c2_linear_gauss_' + error_name + '.svg')
    plt.savefig('c2_linear_gauss_' + error_name + '.png')
    plt.savefig('c2_linear_gauss_' + error_name + '.pdf')

In [ ]:
for (error_name, error_label) in zip(error_name_list,error_label_list):
    fig = plt.figure()
    ax = plt.axes()
    for i in range(len(order_list)):
        x = elem_size_list
        y = classic[i][str(error_name)]

        line, = ax.plot(x, y, '-x', label=(r'$p$ = ' + str(order_list[i])))

        # Calculate the gradient of the line
        gradient, _ = np.polyfit(np.log(x), np.log(y), 1)

        # Annotate the plot with the gradient value
        ax.annotate('Gradient = {:.1f}'.format(gradient), 
                    xy=(x[len(x)//2], y.iloc[len(y)//2]), 
                    xycoords='data',
                    xytext=(-1, 0),  # move a bit to the left
                    textcoords='offset points',
                    ha='right',  # Horizontal alignment is right
                    va='bottom',  # Vertical alignment is bottom
                    )
        ax.set_xscale('log')
        ax.legend(loc='best')
        ax.set_yscale('log')
        ax.set_ylabel(error_label)
        ax.set_xlabel("Element size")
        ax.grid(True, ls=':')

    plt.tight_layout()
    plt.savefig('c2_linear_lenghtEle_' + error_name + '.svg')
    plt.savefig('c2_linear_lenghtEle_' + error_name + '.png')
    plt.savefig('c2_linear_lenghtEle_' + error_name + '.pdf')